In [1]:
import pandas as pd
import polars as pl
import numpy as np
import sys
import re
import requests 
from bs4 import  BeautifulSoup
import json
from datetime import datetime
from tqdm import trange,tnrange, tqdm_notebook
import time

In [101]:
import pandas as pd
import requests
from pandas import json_normalize
import numpy as np
from io import BytesIO
import time
from datetime import datetime, timedelta
import json
import polars as pl


idx = pd.IndexSlice
pd.options.display.float_format = '{:,.3f}'.format


# API request config for SSI API endpoints
ssi_headers = {
        'Connection': 'keep-alive',
        'sec-ch-ua': '"Not A;Brand";v="99", "Chromium";v="98", "Google Chrome";v="98"',
        'DNT': '1',
        'sec-ch-ua-mobile': '?0',
        'X-Fiin-Key': 'KEY',
        'Content-Type': 'application/json',
        'Accept': 'application/json',
        'X-Fiin-User-ID': 'ID',
        'X-Fiin-User-Token':'79,14,109,2,77,63,182,226,168,166,146,27,233,245,5,195,0,200,169,49,153,168,150,254,116,248,73,45,23,61,239,77,183,194,66,44,130,28,239,66,27,56,75,64,56,106,32,203,36,15,41,203,250,254,180,226,198,77,152,213,125,234,19,189,68,11,105,241,12,164,235,22,207,252,226,142,142,239,234,206,6,141,63,201,33,169,127,101,9,25,210,180,120,123,145,86,239,76,144,170,24,236,42,224,97,18,41,250,154,193,169,100,133,44,162,88,201,178,212,112,53,114,12,95,81,58,144,233,119,10,24,138,241,254,39,124,10,203,168,121,230,82,103,254,250,60,20,249,237,230,12,76,78,65,234,48,255,62,102,6,201,137,37,132,182,151,246,26,37,81,123,183,82,209,104,37,195,103,138,251,132,30,158,210,151,131,145,67,235,171,34,119,235,73,193,80,232,193,102,189,47,70,108,12,43,253,60,16,114,169,91,125,8,129,40,14,219,191,133,180,200,249,65,81,108,254,163,47,151,3,1,35,132,124,160,200,104,243,64,185,149,215,77,222,174,177,231,227,14,240,127,167,68,133,199,216',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.102 Safari/537.36',
        'X-Fiin-Seed': 'SEED',
        'sec-ch-ua-platform': 'Windows',
        'Origin': 'https://iboard.ssi.com.vn',
        'Sec-Fetch-Site': 'same-site',
        'Sec-Fetch-Mode': 'cors',
        'Sec-Fetch-Dest': 'empty',
        'Referer': 'https://iboard.ssi.com.vn/',
        'Accept-Language': 'en-US,en;q=0.9,vi-VN;q=0.8,vi;q=0.7'
        }


def organ_listing (lang='vi', headers=ssi_headers):
    """
    Return a DataFrame of all available stock symbols. Live data is retrieved from the SSI API.
    Parameters:
        lang (str): language of the data. Default is 'vi', other options are 'en'
        headers (dict): headers of the request
    """
    url = f"https://fiin-core.ssi.com.vn/Master/GetListOrganization?language={lang}"
    response = requests.request("GET", url, headers=headers)
    status = response.status_code
    if status == 200:
        data = response.json()
        # print('Total number of companies: ', data['totalCount'])
        df = pd.DataFrame(data['items'])
        return df
    else:
        print('Error in API response', response.text)
def financial_report (symbol='SSI', report_type='BalanceSheet', frequency='Quarterly', periods=200, latest_year=None, headers=ssi_headers): # Quarterly, Yearly
    """
    Return financial reports of a stock symbol by type and period.
    Args:
        symbol (:obj:`str`, required): 3 digits name of the desired stock.
        report_type (:obj:`str`, required): BalanceSheet, IncomeStatement, CashFlow
        report_range (:obj:`str`, required): Yearly or Quarterly.
    """
    symbol = symbol.upper()
    organ_code = organ_listing().query(f'ticker == @symbol')['organCode'].values[0]
    this_year = str(datetime.now().year)
    if latest_year == None:
      latest_year = this_year
    else:
      if isinstance(latest_year, int) != True:
        print('Please input latest_year as int number')
      else:
        pass
    url = f'https://fiin-fundamental.ssi.com.vn/FinancialStatement/Download{report_type}?language=vi&OrganCode={organ_code}&Skip=0&Frequency={frequency}&numberOfPeriod={periods}&latestYear={latest_year}'
    response = requests.get(url, headers=headers)
    # print(response.text)
    status = response.status_code
    if status == 200:
        df = pd.read_excel(BytesIO(response.content), skiprows=7)
        # .dropna()
        return df
    else:
        print(f'Error {status} when getting data from SSI. Details:\n {response.text}')
        return None

def mc(symbol='SSI',frequency='Quarterly'):
        headers = {
                'Connection': 'keep-alive',
                'sec-ch-ua': '"Not A;Brand";v="99", "Chromium";v="98", "Google Chrome";v="98"',
                'DNT': '1',
                'sec-ch-ua-mobile': '?0',
                'X-Fiin-Key': 'KEY',
                'Content-Type': 'application/json',
                'Accept': 'application/json',
                'X-Fiin-User-ID': 'ID',
                'X-Fiin-Key':'KEY',
                'X-Fiin-Seed':'SEED',
                'X-Fiin-User-Token':'93,203,163,40,224,188,115,115,138,126,18,199,199,124,39,108,231,125,80,15,79,226,178,184,60,101,162,174,35,156,160,54,113,153,99,49,167,98,81,217,225,67,146,16,255,228,25,242,213,192,129,186,139,181,191,112,119,41,36,49,45,37,208,216,184,215,157,52,95,29,185,63,186,228,97,27,86,163,49,131,67,17,92,172,156,132,217,88,15,231,7,175,164,138,29,180,116,130,76,38,107,88,132,186,75,8,124,209,185,88,180,7,211,235,229,42,232,206,219,25,84,76,226,0,197,66,181,79,230,74,208,200,86,229,25,9,26,44,219,167,162,161,178,144,90,239,165,36,41,99,186,205,217,181,7,162,101,238,186,34,56,31,153,19,176,193,110,47,18,237,192,133,113,67,194,227,13,202,239,126,23,189,121,36,77,74,211,188,203,144,113,209,48,248,84,22,237,223,154,232,228,74,124,239,104,86,146,26,237,250,25,53,58,197,214,59,195,119,6,146,131,42,111,254,70,220,1,233,163,205,29,132,94,248,229,177,175,42,32,130,189,20,83,218,153,0,143,234,246,130,103,169,144,137,128,169',
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.102 Safari/537.36',
                'X-Fiin-Seed': 'SEED',
                'sec-ch-ua-platform': 'Windows',
                'Origin': 'https://iboard.ssi.com.vn',
                'Sec-Fetch-Site': 'same-site',
                'Sec-Fetch-Mode': 'cors',
                'Sec-Fetch-Dest': 'empty',
                'Referer': 'https://iboard.ssi.com.vn/',
                'Accept-Language': 'en-US,en;q=0.9,vi-VN;q=0.8,vi;q=0.7'
                }
        symbol = symbol.upper()
        organ_code = organ_listing().query(f'ticker == @symbol')['organCode'].values[0]
        url = f'https://fiin-fundamental.ssi.com.vn/FinancialAnalysis/DownloadFinancialRatio2?language=vi&OrganCode={organ_code}&CompareToIndustry=false&Frequency={frequency}&Ratios=ryd11&TimeLineFrom=2001&TimeLineTo=2025'
        response = requests.get(url, headers=headers)
        status = response.status_code
        if status == 200:
            df = pd.read_excel(BytesIO(response.content), skiprows=7)
            df = df.rename(columns={'Chỉ số': 'LengthReport'})
            return df.iloc[[1],:]
        else:
            print(f'Error {status} when getting data. Details:\n {response.text}')
            return None



def get_mc_Y(ticker):    
    x = mc(ticker,'Yearly')
    x = x.T
    x.columns = x.iloc[0]
    x = x.iloc[1:,:]
    #include only rows contains ".3" in last 2 string of index
    x = x[x.index.str[-2:] == '.3']
    #get first 4 characters of index
    x.index = x.index.str[:4]
    x['dates'] = x.index.astype(int)
    #replace 0 with NaN value
    x = x.replace(0, np.nan)
    #rename column 2 to 'mc'
    x = x.rename(columns={x.columns[0]: 'mc'})
    return x

In [83]:
x = mc('BCM','Yearly')
x = x.T
y = mc('BCM','Quarterly')
y = y.T

In [105]:
y

,1
LengthReport,BCM
Q1 2018,NaN
Q2 2018,"15,487,773,120,000.000"
Q3 2018,NaN
Q4 2018,"23,825,020,701,900.000"
Q1 2019,"34,183,980,000,000.000"
Q2 2019,"31,372,920,000,000.000"
Q3 2019,"30,118,500,000,000.000"
Q4 2019,"24,089,625,000,000.000"
Q1 2020,"26,859,285,000,000.000"


In [88]:
y

,1
LengthReport,BCM
Q1 2018,NaN
Q2 2018,"15,487,773,120,000.000"
Q3 2018,NaN
Q4 2018,"23,825,020,701,900.000"
Q1 2019,"34,183,980,000,000.000"
Q2 2019,"31,372,920,000,000.000"
Q3 2019,"30,118,500,000,000.000"
Q4 2019,"24,089,625,000,000.000"
Q1 2020,"26,859,285,000,000.000"
